In [1]:
%load_ext kedro.ipython

[07/01/24 15:41:23] INFO     Registered line magic '%reload_kedro'                                   ]8;id=121492;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=567832;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=407062;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=149380;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#60\60]8;;\

                    INFO     Resolved project path as: /Users/wilsvenleong/Downloads/Learning       ]8;id=73968;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=933881;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#171\171]8;;\
                             Materials/healthhub-content-optimization/content-optimization.                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[07/01/24 15:41:24] INFO     Kedro project Content Optimization                                     ]8;id=857917;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=960870;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=565166;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=325159;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=715093;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=591872;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [2]:
import re

import pandas as pd

In [3]:
# ruff: noqa: F821
catalog.list()


[
    'all_contents',
    'all_contents_standardized',
    'all_contents_extracted',
    'all_extracted_text',
    'merged_data',
    'raw_word_counts',
    'log_word_counts',
    'flag_for_removal_by_type',
    'parameters',
    'params:columns_to_keep',
    'params:columns_to_keep.cost-and-financing',
    'params:columns_to_keep.diseases-and-conditions',
    'params:columns_to_keep.health-statistics',
    'params:columns_to_keep.live-healthy-articles',
    'params:columns_to_keep.medical-care-and-facilities',
    'params:columns_to_keep.medications',
    'params:columns_to_keep.program-sub-pages',
    'params:columns_to_keep.programs',
    'params:columns_to_keep.support-group-and-others',
    'params:columns_to_add',
    'params:columns_to_add.health-statistics',
    'params:columns_to_add.medications',
    'params:columns_to_add.program-sub-pages',
    'params:columns_to_add.programs',
    'params:default_columns',
    'params:word_count_cutoff'
]

In [4]:
# ruff: noqa: F821
merged_data = catalog.load("merged_data")
merged_data

                    INFO     Loading data from merged_data (ParquetDataset)...                  ]8;id=732570;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=990877;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"<div class=""ExternalClass07C58E0D957B4AA7B14FC...",...,0.216244,0.216244,cost-and-financing,False,True,False,[Cancer Facts You Cannot Ignore],"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/marri...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,"<div class=""ExternalClassE1D82270F17241E495537...",...,0.111180,0.327423,cost-and-financing,False,True,False,"[MediSave, Baby Bonus What You Need to Know, I...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/medisave,medisave,MediSave is the national medical savings schem...,"<div class=""ExternalClass67AD25F1F8B64B349E515...",...,0.063847,0.391271,cost-and-financing,False,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...","What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/finan...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,"<div class=""ExternalClassE335708125E743FDAA331...",...,0.061297,0.452568,cost-and-financing,False,False,False,[],"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/chas,chas,"With a CHAS card, all Singapore citizens can r...",<h2>What is the Community Health Assist Scheme...,...,0.060281,0.512849,cost-and-financing,False,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,transitional-care-programme-for-heart-failure,The team from NUHCS gives support to heart fai...,"<div class=""ExternalClassFC126593610D4F0587A4B...",...,0.035071,0.880618,support-group-an

In [5]:
merged_data["content_category"].value_counts().sort_values(ascending=False)


content_category
live-healthy-articles          1155
medications                     579
program-sub-pages               350
diseases-and-conditions         325
programs                         89
medical-care-and-facilities      59
cost-and-financing               27
health-statistics                15
support-group-and-others         14
Name: count, dtype: int64

In [6]:
merged_data.groupby(by="content_category")["to_remove"].value_counts().loc[
    :, True
].sort_values(ascending=False)


content_category
program-sub-pages              213
programs                        70
live-healthy-articles           45
diseases-and-conditions         16
health-statistics                8
medications                      7
cost-and-financing               4
medical-care-and-facilities      2
support-group-and-others         2
Name: count, dtype: int64

In [7]:
(
    merged_data.groupby(by="content_category")["to_remove"]
    .value_counts(normalize=True)
    .loc[:, True]
    .sort_values(ascending=False)
    * 100.00
).round(2)


content_category
programs                       78.65
program-sub-pages              60.86
health-statistics              53.33
cost-and-financing             14.81
support-group-and-others       14.29
diseases-and-conditions         4.92
live-healthy-articles           3.90
medical-care-and-facilities     3.39
medications                     1.21
Name: proportion, dtype: float64

## NaN

In [8]:
merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: x.isna().sum()
).sort_values(ascending=False)


content_category
program-sub-pages              6
cost-and-financing             2
programs                       2
medical-care-and-facilities    1
diseases-and-conditions        0
health-statistics              0
live-healthy-articles          0
medications                    0
support-group-and-others       0
Name: content_body, dtype: int64

In [9]:
content_category = "cost-and-financing"

merged_data.query(f"content_category == '{content_category}' and content_body.isna()")

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
24,1468064,1Cost and Financing,1Cost and Financing,None,None,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/1Cost...,1Cost-and-Financing,None,None,...,0.0,1.0,cost-and-financing,True,None,None,None,None,None,None
26,1468209,1testcnf2,1testcnf2,None,None,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/1test...,1testcnf2,None,None,...,0.0,1.0,cost-and-financing,True,None,None,None,None,None,None


## Excel Error

In [10]:
merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: (x.str.contains("Value exceeded maximum cell")).sum()
).sort_values(ascending=False)


content_category
program-sub-pages              125
programs                        28
diseases-and-conditions          5
live-healthy-articles            4
cost-and-financing               0
health-statistics                0
medical-care-and-facilities      0
medications                      0
support-group-and-others         0
Name: content_body, dtype: int64

In [11]:
content_category = "live-healthy-articles"

# Define the regex pattern
pattern = re.compile(r"(Value exceeded maximum cell)")


# Function to apply regex and return True or False
def apply_regex_v1(x):
    if pd.isna(x):
        return False
    return bool(pattern.search(str(x)))


filtered = merged_data[merged_data["content_body"].apply(lambda x: apply_regex_v1(x))]
filtered.query(f"content_category == '{content_category}'")

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
372,1446016,Nutrition During Pregnancy—Eating Right for Two,Nutrition During Pregnancy—Eating Right for Two,None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/pregnanc...,www.healthhub.sg/live-healthy/pregnancy-nutrit...,pregnancy-nutrition-during-pregnancy-eating-ri...,Can you have a healthy weight gain during preg...,Value exceeded maximum cell size|MaximumCellSi...,...,0.010393,0.082520,live-healthy-articles,True,None,None,None,None,None,None
378,1444820,Recommended Dietary Allowances,Recommended Dietary Allowances,"Food and Nutrition,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/recommen...,www.healthhub.sg/live-healthy/recommended_diet...,recommended_dietary_allowances,The Recommended Dietary Allowances (RDA) refer...,Value exceeded maximum cell size|MaximumCellSi...,...,0.007304,0.130105,live-healthy-articles,True,None,None,None,None,None,None
403,1445643,Getting Support (for STIs),Getting Support (for STIs),"Sexual Health and Relationships,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/gettings...,www.healthhub.sg/live-healthy/gettingsupportstis,gettingsupportstis,Do you have questions about sex or sexually tr...,Value exceeded maximum cell size|MaximumCellSi...,...,0.004092,0.261496,live-healthy-articles,True,None,None,None,None,None,None
625,1446045,Help Your Child Cope With Puberty and Self-esteem,Help Your Child Cope With Puberty and Self-esteem,"Mind and Balance,Sexual Health and Relationships,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/live-healthy/help-you...,www.healthhub.sg/live-healthy/help-your-child-...,help-your-child-cope-with-puberty-and-self-esteem,Help your children prepare for this important ...,Value exceeded maximum cell size|MaximumCellSi...,...,0.000983,0.670407,live-healthy-articles,True,None,None,None,None,None,None


In [12]:
filtered.groupby(by="content_category").size().sort_values(ascending=False)


content_category
program-sub-pages          125
programs                    28
diseases-and-conditions      5
live-healthy-articles        4
dtype: int64

In [13]:
# Either NaN or Value exceeded maximum cell
merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: (x.isna() | x.str.contains("Value exceeded maximum cell")).sum()
).sort_values(ascending=False)


content_category
program-sub-pages              131
programs                        30
diseases-and-conditions          5
live-healthy-articles            4
cost-and-financing               2
medical-care-and-facilities      1
health-statistics                0
medications                      0
support-group-and-others         0
Name: content_body, dtype: int64

## No Tags

In [14]:
# Define the regex pattern
pattern = re.compile(r"(<[div|p|h2].*?>)")


# Function to apply regex and return True or False
def apply_regex_v2(x):
    if pd.isna(x) or "Value exceeded maximum cell" in str(x):
        return True
    return bool(pattern.search(str(x)))


# Group by content_category and apply the regex
grouped = merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: ~x.apply(apply_regex_v2)
)

grouped.groupby(by="content_category").value_counts().loc[:, True].sort_values(
    ascending=False
)


content_category
program-sub-pages          34
programs                    5
live-healthy-articles       2
cost-and-financing          1
diseases-and-conditions     1
Name: count, dtype: int64

## No Extracted Content

In [15]:
merged_data.groupby(by="content_category")["extracted_content_body"].apply(
    lambda x: (x == "").sum()
).sort_values(ascending=False)


content_category
programs                       22
program-sub-pages               8
live-healthy-articles           1
support-group-and-others        1
cost-and-financing              0
diseases-and-conditions         0
health-statistics               0
medical-care-and-facilities     0
medications                     0
Name: extracted_content_body, dtype: int64

In [16]:
content_category = "program-sub-pages"

merged_data.query(
    f"content_category == '{content_category}' and extracted_content_body == ''"
)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
2227,1503039,what-are-vapes,what-are-vapes,None,None,https://www.healthhub.sg/programmes/IQuit/e-ci...,www.healthhub.sg/programmes/IQuit/e-cig/what-a...,what-are-vapes,None,"<div style=""opacity: 0;"">What are vapes page i...",...,0.001705,0.910522,program-sub-pages,True,False,False,[],[],[],
2243,1503042,what-s-in-the-news,what-s-in-the-news,None,None,https://www.healthhub.sg/programmes/IQuit/e-ci...,www.healthhub.sg/programmes/IQuit/e-cig/what-s...,what-s-in-the-news,None,"<div style=""opacity: 0;"">What's in the news is...",...,0.001299,0.933910,program-sub-pages,True,False,False,[],[],[],
2258,1434909,Let’s talk about Vascular Dementia_dementia_Le...,Let’s talk about Vascular Dementia,None,None,https://www.healthhub.sg/programmes/AAP/dementia/,www.healthhub.sg/programmes/AAP/dementia/,dementia,Vascular dementia is one of the most common fo...,"<div class=""ExternalClass2E55A9B34B7E47CC926CD...",...,0.001015,0.951042,program-sub-pages,True,False,False,[],[],[],
2364,1434913,MindSG_Caring-For-Ourselves_Level1,MindSG,None,None,https://www.healthhub.sg/programmes/MindSG/Car...,www.healthhub.sg/programmes/MindSG/Caring-For-...,Caring-For-Ourselves,Explore our suite of self-care tools and resou...,"<div class=""ExternalClass6A175BA8870B4B51BA62A...",...,0.000040,1.000000,program-sub-pages,True,False,False,[],[],[],
2374,1435024,Test page_test-page_Level2,Test page,None,None,https://www.healthhub.sg/programmes/parent-hub...,www.healthhub.sg/programmes/parent-hub/activit...,test-page,javascript testing,"<img src=""https://ch-api.healthhub.sg/api/publ...",...,0.000000,1.000000,program-sub-pages,True,False,True,[],[],[],
2414,1434841,MindSG_Caring-For-Others_Level1,MindSG,None,None,https://www.healthhub.sg/programmes/MindSG/Car...,www.healthhub.sg/programmes/MindSG/Caring-For-...,Caring-For-Others,Explore our suite of self-care tools and resou...,"<div class=""ExternalClass951331D026A741A083582...",...,0.000000,1.000000,program-sub-pages,True,False,False,[],[],[],
2436,1434805,Dr Kimberly Douglas_dr-kimberly-douglas_Level3,Dr Kimberly Douglas,None,None,https://www.healthhub.sg/programmes/healthia/d...,www.healthhub.sg/programmes/healthia/dr-kimber...,dr-kimberly-douglas,Healthia - Dr Kimberly Douglas,"<div class=""ExternalClass84A6D93A5F144F208BA91...",...,0.000000,1.000000,program-sub-pages,True,False,True,[],"[[, /programs/Lists/Program Sub Pages/index.ht...","[[Gallery, h4], [Elements, h4], [Pages, h4], [...",
2451,1434777,About Us_about-us_Level1,About Us,None,None,https://www.healthhub.sg/programmes/healthia/a...,www.healthhub.sg/programmes/healthia/about-us/,about-us,Healthia - About Us,"<div class=""ExternalClassC2A19274671044F2A00D1...",...,0.000000,1.000000,program-sub-pages,True,False,True,[],"[[, /programs/Lists/Program Sub Pages/index.ht...","[[Gallery, h4], [Elements, h4], [Pages, h4], [...",


## Below Word Count

In [17]:
# ruff: noqa: F821
word_count_cutoff = catalog.load("params:word_count_cutoff")
word_count_cutoff

[07/01/24 15:41:25] INFO     Loading data from params:word_count_cutoff (MemoryDataset)...      ]8;id=984242;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=611772;file:///opt/anaconda3/envs/content-opt/lib/python3.12/site-packages/kedro/io/data_catalog.py#508\508]8;;\

90

In [18]:
tmp = merged_data.query("extracted_content_body.notna()")
tmp = tmp[
    tmp["extracted_content_body"].apply(
        lambda x: len(x.split()) > 0 and len(x.split()) <= word_count_cutoff
    )
]
tmp.groupby(by="content_category").size().sort_values(ascending=False)


content_category
program-sub-pages              40
live-healthy-articles          38
programs                       13
diseases-and-conditions        10
health-statistics               8
medications                     7
cost-and-financing              1
medical-care-and-facilities     1
support-group-and-others        1
dtype: int64

---

## Examples (Excel Error)

In [19]:
content_category = "diseases-and-conditions"

tmp = merged_data.query(
    f"content_category == '{content_category}' and content_body.str.contains('Value exceeded maximum cell')"
).reset_index(drop=True)
tmp

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
0,1437331,HIV and AIDS,HIV and AIDS,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/t...,topics_hiv_aids,"With early and effective treatment, people liv...",Value exceeded maximum cell size|MaximumCellSi...,...,0.001856,0.838698,diseases-and-conditions,True,None,None,None,None,None,None
1,1464135,Returning to Work,Social and Leisure Matters after Stroke: Retur...,None,None,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/s...,stroke_returning_to_work,Post-stroke management tips on returning to wo...,Value exceeded maximum cell size|MaximumCellSi...,...,0.000505,0.994560,diseases-and-conditions,True,None,None,None,None,None,None
2,1464154,Returning to Work (Chinese),中风后的社会及休闲活动: 中风后重返职场,None,None,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/s...,stroke_returning_to_work_chinese,Post-stroke management on returning to work,Value exceeded maximum cell size|MaximumCellSi...,...,0.000419,0.997360,diseases-and-conditions,True,None,None,None,None,None,None
3,1471296,Returning to work (Tamil),பக்கவாதம் ஏற்பட்டதற்குப் பிறகு உள்ள சமூக மற்று...,None,None,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/s...,stroke_returning_to_work_tamil,Post-stroke tips on returning to work,Value exceeded maximum cell size|MaximumCellSi...,...,0.000398,0.997758,diseases-and-conditions,True,None,None,None,None,None,None
4,1464163,Returning to Work (Malay),Kegiatan Sosial dan Masa Lapang selepas Angin ...,None,None,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/s...,stroke_returning_to_work_malay,Post-stroke management tip in returning in work,Value exceeded maximum cell size|MaximumCellSi...,...,0.000358,0.998486,diseases-and-conditions,True,None,None,None,None,None,None


In [20]:
tmp["content_body"].values.tolist()


[
    'Value exceeded maximum cell size|MaximumCellSizeReached',
    'Value exceeded maximum cell size|MaximumCellSizeReached',
    'Value exceeded maximum cell size|MaximumCellSizeReached',
    'Value exceeded maximum cell size|MaximumCellSizeReached',
    'Value exceeded maximum cell size|MaximumCellSizeReached'
]

In [21]:
tmp["title"].values.tolist()


[
    'HIV and AIDS',
    'Social and Leisure Matters after Stroke: Returning to Work',
    '中风后的社会及休闲活动: 中风后重返职场',
    'பக்கவாதம் ஏற்பட்டதற்குப் பிறகு உள்ள சமூக மற்றும் பொழுதுபோக்கு விவகாரங்கள்: வேலைக்குத் திரும்புதல்',
    'Kegiatan Sosial dan Masa Lapang selepas Angin Ahmar: Kembali Bekerja Selepas Angin Ahmar'
]

In [22]:
tmp["full_url"].values.tolist()


[
    'https://www.healthhub.sg/a-z/diseases-and-conditions/topics_hiv_aids',
    'https://www.healthhub.sg/a-z/diseases-and-conditions/stroke_returning_to_work',
    'https://www.healthhub.sg/a-z/diseases-and-conditions/stroke_returning_to_work_chinese',
    'https://www.healthhub.sg/a-z/diseases-and-conditions/stroke_returning_to_work_tamil',
    'https://www.healthhub.sg/a-z/diseases-and-conditions/stroke_returning_to_work_malay'
]

## Examples (No Tags)

In [23]:
content_category = "cost-and-financing"

tmp = merged_data.query(
    f"content_category == '{content_category}' and content_body.notna()"
)
tmp = tmp[~tmp["content_body"].apply(lambda x: apply_regex_v2(x)).values].reset_index(
    drop=True
)
tmp

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
0,1468614,1test-2,Test Entry,None,None,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/1test-2,1test-2,Sagittis aliquam malesuada bibendum arcu vitae...,Pellentesque habitant morbi tristique senectus...,...,0.0,1.0,cost-and-financing,True,None,None,None,None,None,None


In [24]:
print(tmp["content_body"].values.tolist()[0])

Pellentesque habitant morbi tristique senectus et netus et malesuada fames. Sodales neque sodales ut etiam sit amet nisl purus. Risus at ultrices mi tempus imperdiet nulla. Euismod in pellentesque massa placerat duis ultricies lacus sed.


In [25]:
tmp["title"].values.tolist()

['Test Entry']

In [26]:
tmp["full_url"].values.tolist()

['https://www.healthhub.sg/a-z/costs-and-financing/1test-2']

## Examples (No Extracted Content)

In [27]:
content_category = "support-group-and-others"

tmp = merged_data.query(
    f"content_category == '{content_category}' and extracted_content_body == ''"
)
tmp

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
2603,1440793,HSA: Search for Registered Health Products,HSA: Search for Registered Health Products,None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,registered-health-products,Find out if your Medicine/Medical Devices/Chin...,"<div class=""ExternalClass7EA11F1C5E404F449FD8A...",...,0.065381,0.668331,support-group-and-others,True,False,False,[],"[[Find Out, https://www.healthhub.sg/programme...",[],


In [28]:
tmp["content_body"].values.tolist()


[
    '<div class="ExternalClass7EA11F1C5E404F449FD8A9A5D1ED4CF0"><p> \n      <br> \n   </p><p>\n      <strong style="text-decoration:underline;color:#43afff;"> \n         <a href="https://www.healthhub.sg/programmes/hsa" style="text-decoration:underline;">Find\xa0Out</a></strong><strong style="color:#43afff;"> </strong>if your Medicine/Medical Devices/Chinese Proprietary Medicines are registered/listed in Singapore<br></p><p>\n      <br>\n   </p></div>\n<br>\n<br>'
]

In [29]:
tmp["title"].values.tolist()

['HSA: Search for Registered Health Products']

In [30]:
tmp["full_url"].values.tolist()

['https://www.healthhub.sg/a-z/support-groups-and-others/registered-health-products']

## Examples (Below Word Count)

In [31]:
tmp = merged_data.query("extracted_content_body.notna()")
tmp = tmp[
    tmp["extracted_content_body"].apply(
        lambda x: len(x.split()) > 0 and len(x.split()) <= word_count_cutoff
    )
]

In [32]:
content_category = "diseases-and-conditions"

sampled = tmp.query(f"content_category == '{content_category}'").sample(1)

print(sampled.index[0])
print(sampled["full_url"].values[0])
print(sampled["title"].values[0])
print(sampled["extracted_content_body"].values[0])

339
https://www.healthhub.sg/a-z/diseases-and-conditions/1test
EY Test Entry
Test Entry - 15/11/2023


In [33]:
selected_content_categories = [
    "live-healthy-articles",
    "support-group-and-others",
    "diseases-and-conditions",
    "cost-and-financing",
    "medical-care-and-facilities",
]

to_save = tmp.query(f"content_category.isin({selected_content_categories})")
to_save.head()

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,percentage_total_views,cumulative_percentage_total_views,content_category,to_remove,has_table,has_image,related_sections,extracted_links,extracted_headers,extracted_content_body
7,1435035,Hospital Bills Estimates in Singapore,Hospital Bills Estimates in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/hospi...,hospital-charges,Manage your healthcare costs by assessing the ...,"<div class=""ExternalClass3990B681758C432A8A905...",...,0.047452,0.660883,cost-and-financing,True,False,False,[],[[https://moh.gov.sg/cost-financing/bill-estim...,[],The Ministry of Health publishes hospital bill...
125,1437417,Hearing Tests,Hearing Tests,None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/h...,hearing_test_sgh,A basic hearing test or pure tone audiometry i...,"<div class=""ExternalClassA79F25C288FF433383646...",...,0.002389,0.767962,diseases-and-conditions,True,False,False,[],"[[click here, http://www.singhealth.com.sg/Pat...","[[​Overview , h2]]",Overview\nA basic hearing test or pure tone au...
182,1437667,Colonoscopy Procedure,Colonoscopy Procedure,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/c...,colonoscopy-video,What is a colonoscopy? Get an in-depth look by...,"<div class=""ExternalClass3273C8E7BACB45EE87C06...",...,0.001486,0.876739,diseases-and-conditions,True,False,False,[],[],"[[What is a Colonoscopy?\n, h2]]",What is a Colonoscopy?\nA colonoscopy is a pro...
297,1437297,Allergies (Child),Allergies (Child),None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/a...,allergies_child_sgh,Find out what are the common childhood allergi...,"<div class=""ExternalClass1FE95FFA9E1D4B2AAC250...",...,0.000581,0.984935,diseases-and-conditions,True,False,False,"[Conjuctivitis, Ringworm, Cough]","[[click here, http://www.singhealth.com.sg/Pat...","[[Overview, h2]]",OverviewAllergies are some of the most common ...
308,1437335,Chronic Rhinitis,Chronic Rhinitis,None,https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/diseases-and-cond...,www.healthhub.sg/a-z/diseases-and-conditions/c...,chronic_rhinitis,Chronic rhinitis causes mucus accumulation and...,"<div class=""ExternalClass4AD22DD1314C45BA95EA6...",...,0.000521,0.990982,diseases-and-conditions,True,False,False,[],[],[],The cavities around nasal passages (sinuses) b...


## Save Excel with `type` column for VML

In [34]:
merged_data.shape

(2613, 33)

In [35]:
merged_data["type"] = None

In [36]:
na_indexes = merged_data[merged_data["content_body"].isna()].index
excel_error_indexes = merged_data.query(
    "content_category.notna() and content_body.str.contains('Value exceeded maximum cell')"
).index
no_tags_indexes = merged_data[
    ~merged_data.query("content_category.notna()")["content_body"].apply(
        lambda x: apply_regex_v2(x)
    )
].index
no_extracted_content = merged_data[merged_data["extracted_content_body"] == ""].index

# Assuming merged_data is your DataFrame and word_count_cutoff is defined
indices = merged_data.query("extracted_content_body.notna()")[
    "extracted_content_body"
].apply(lambda x: len(x.split()) > 0 and len(x.split()) <= word_count_cutoff)
# Get the indices of the True values
below_word_count_indexes = indices[indices].index

In [37]:
merged_data.loc[na_indexes, "type"] = "NaN"
merged_data.loc[excel_error_indexes, "type"] = "Excel Error"
merged_data.loc[no_tags_indexes, "type"] = "No HTML Tags"
merged_data.loc[no_extracted_content, "type"] = "No Extracted Content"
merged_data.loc[below_word_count_indexes, "type"] = "Below Word Count"

In [38]:
merged_data["type"].value_counts()


type
Excel Error             162
Below Word Count        119
No HTML Tags             43
No Extracted Content     32
NaN                      11
Name: count, dtype: int64

In [39]:
merged_data.columns


Index(['id', 'content_name', 'title', 'article_category_names',
       'cover_image_url', 'full_url', 'full_url2', 'friendly_url',
       'category_description', 'content_body', 'keywords', 'feature_title',
       'pr_name', 'alternate_image_text', 'date_modified', 'number_of_views',
       'last_month_view_count', 'last_two_months_view', 'page_views',
       'engagement_rate', 'bounce_rate', 'exit_rate', 'scroll_percentage',
       'percentage_total_views', 'cumulative_percentage_total_views',
       'content_category', 'to_remove', 'has_table', 'has_image',
       'related_sections', 'extracted_links', 'extracted_headers',
       'extracted_content_body', 'type'],
      dtype='object')

In [40]:
to_save = merged_data[
    [
        "id",
        "content_name",
        "title",
        "full_url",
        "content_body",
        "content_category",
        "to_remove",
        "type",
    ]
]

In [41]:
# ruff: noqa: F821
catalog.save("flag_for_removal_by_type", to_save)